In [1]:
import re
import pickle 
import os 
import glob
from tqdm import tqdm
import pympi.Elan 
import pandas as pd
from sklearn.preprocessing import StandardScaler

import torch 
from transformers import BertModel 
from transformers import BertJapaneseTokenizer

import utils

from transformers import logging
logging.set_verbosity_error()


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
feature_path = '../data/Hazumi_features/'

# オークナイザーの読み込み
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking"
)

# 学習済みモデルの読み込み
model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking").to(device)


def preprocess(text):
    """ 
    フィラー記号(F *)，|の除去
    """
    text = re.sub('\([^)]*\)', '', text)
    text = text.replace('|', '')

    return text


def embedding(sentences):
    # 前処理

    sentences = [preprocess(text) for text in sentences]

    # BERTトークン化
    encoded = tokenizer.batch_encode_plus(
        sentences, padding=True, add_special_tokens=True
    )


    # BERTトークンID列を抽出
    input_ids = torch.tensor(encoded["input_ids"], device=device) 

    # BERTの最大許容トークン数が512なので超える場合は切り詰める
    input_ids = input_ids[:, :512] 

    with torch.no_grad():
        outputs = model(input_ids) 

    # 最終層の隠れ状態ベクトルを取得
    last_hidden_states = outputs[0] 

    # [CLS]トークンの単語ベクトルを抽出
    vecs = last_hidden_states[:, 0, :]

    return vecs.tolist()


def eaf_to_df( eaf: pympi.Elan.Eaf ) -> pd.DataFrame:
    tier_names = list( eaf.tiers.keys() )

    def timeslotid_to_time( timeslotid: str ) -> float:
        return eaf.timeslots[ timeslotid ] / 1000

    def parse( tier_name: str, tier: dict ) -> pd.DataFrame:
        values = [ (key,) + value[:-1] for key, value in tier.items() ]
        df = pd.DataFrame( values, columns=[ "id", "start", "end", "transcription"] )

        df["start"] = df["start"].apply( timeslotid_to_time )
        df["end"] = df["end"].apply( timeslotid_to_time )
        df["ID"] = df.apply( lambda x: f"{tier_name}-{x.name}", axis=1 )
        df = df.reindex( columns=["ID", "start", "end", "transcription"] )

        return df

    dfs = [ parse(tier_name=name, tier=eaf.tiers[name][0]) for name in tier_names ]
    df = pd.concat( dfs )
    df = df.sort_values( "start" )
    df = df.reset_index( drop=True )
    return df

def extract_sentence(version, filename, start):
    res = []
    sys_res = []
    raw_sentence = []
    raw_sys_sentence = []
    src = f'../data/Hazumi/Hazumi{version}/elan/' + filename + '.eaf' 

    eaf = pympi.Elan.Eaf(src) 
    df = eaf_to_df(eaf) 

    df['start'] = (df['start'] * 1000).astype(int)


    for time in start:
        sentence = df[(df['start'] == time) & (df['ID'].str.contains('user'))]['transcription'].values.tolist()
        sys_sentence = df[(df['start'] == time) & (df['ID'].str.contains('sys'))]['transcription'].values.tolist()
        if len(sentence) == 0:
            sentence = ['']
        if len(sys_sentence) == 0:
            sys_sentence = ['']
        raw_sentence.append(sentence) 
        raw_sys_sentence.append(sys_sentence)
        sentence = embedding(sentence)
        sys_sentence = embedding(sys_sentence)
        res.append(sentence[0])
        sys_res.append(sys_sentence[0])

    return res


def personality_binarization(x, vid):
    """
    性格特性スコアを2クラスに分類
    """
    binary = {}
    df = pd.DataFrame.from_dict(x, orient='index')

    df = (df >= 0) * 1

    for id in vid:
        binary[id] = df.loc[id, :].tolist()

    return binary

def get_personality(hazumiversion, filename):
    """
    アノテータの質問回答から性格特性スコアを算出
    """
    if hazumiversion == "1911":
        filepath = "../data/Hazumi/Hazumi1911/questionnaire/1911questionnaires.xlsx"
        df = pd.read_excel(filepath, sheet_name=4, header=1, index_col=0)
    elif hazumiversion == "2010":
        filepath = "../data/Hazumi/Hazumi2010/questionnaire/questionnaires.xlsx"
        df = pd.read_excel(filepath, sheet_name=3, header=1, index_col=0)
    elif hazumiversion == "2012":
        filepath = "../data/Hazumi/Hazumi2012/questionnaire/questionnaires.xlsx"
        df = pd.read_excel(filepath, sheet_name=3, header=1, index_col=0)
    data = df.loc[filename].values.tolist()
    return [data[0] + (8 - data[5]), (8-data[1]) + data[6], data[2] + (8-data[7]), data[3] + (8-data[8]), data[4] + (8-data[9])]

def get_third_personality(version, filename):
    """
    アノテータの質問回答から性格特性スコアを算出
    """
    if version == "1911":
        file_path = '../data/Hazumi/Hazumi1911/questionnaire/220818thirdbigfive-Hazumi1911.xlsx'
    elif version == "2010":
        file_path = '../data/Hazumi/Hazumi2010/questionnaire/220822thirdbigfive-Hazumi2010.xlsx'
    df = pd.read_excel(file_path, sheet_name=5, header=1, index_col=0)
    data = df.loc[filename].values.tolist()
    return [data[5], data[13], data[21], data[29], data[37]]

/home/ryoyanagimoto/anaconda3/envs/personality/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# 1712
version = "1712"
dumpfile_path = f'../data/Hazumi/Hazumi{version}/dumpfiles/*'
files = glob.glob(dumpfile_path)

text = {}
audio = {}
visual = {} 

SP = {}
TP = {}

SS = {}
TS = {}

vid = []

for file_path in tqdm(sorted(files)):
    filename = os.path.basename(file_path).split('.', 1)[0]
    df = pd.read_csv(file_path)
    start = df['start(exchange)[ms]'].values.tolist()

    vid.append(filename)
    text[filename] = extract_sentence(version, filename, start)
    audio[filename] = df.loc[:, 'pcm_RMSenergy_sma_max':'F0_sma_de_kurtosis'].values.tolist()
    visual[filename] = df.loc[:, '17_acceleration_max':'AU45_c_mean'].values.tolist()

    # SP[filename] = get_personality(version, filename)
    # TP[filename] = get_third_personality(version, filename)

    # row
    # SS[filename] = df.loc[:, 'SS'].astype(int).values.tolist()
    # TS[filename] = (df.loc[:, 'TS1':'TS5'].mean(axis='columns') > 4.0).astype(int).values.tolist()
    # binary
    # SS[filename] = (df.loc[:, 'SS'] > 4.0).astype(int).values.tolist()
    TS[filename] = (df.loc[:, 'TS1':'TS5'].mean(axis='columns') > 4.0).astype(int).values.tolist()
    # ternary
    # SS[filename] = df.loc[:, 'SS_ternary'].astype(int).values.tolist()
    # TS[filename] = df.loc[:, 'TS_ternary'].astype(int).values.tolist()

# SP_scaled = utils.dict_standardize(SP, vid)
# SP_binary = personality_binarization(SP_scaled, vid)
# SP_cluster = utils.clustering(SP_scaled, vid, n_clusters=4)

with open(feature_path + f'Hazumi{version}_features.pkl', mode='wb') as f:
    pickle.dump((SS, TS, SP, TP, text, audio, visual, vid), f)

  0%|          | 0/27 [00:00<?, ?it/s]

100%|██████████| 27/27 [00:35<00:00,  1.30s/it]


In [10]:
# 1902
version = "1902"
dumpfile_path = f'../data/Hazumi/Hazumi{version}/dumpfiles/*'
files = glob.glob(dumpfile_path)

text = {}
audio = {}
visual = {} 

SP = {}
TP = {}

SS = {}
TS = {}

vid = []

for file_path in tqdm(sorted(files)):
    filename = os.path.basename(file_path).split('.', 1)[0]
    df = pd.read_csv(file_path)
    start = df['start(exchange)[ms]'].values.tolist()

    vid.append(filename)
    text[filename] = extract_sentence(version, filename, start)
    audio[filename] = df.loc[:, 'pcm_RMSenergy_sma_max':'F0_sma_de_kurtosis'].values.tolist()
    visual[filename] = df.loc[:, '17_acceleration_max':'AU45_c_mean'].values.tolist()

    # SP[filename] = get_personality(version, filename)
    # TP[filename] = get_third_personality(version, filename)

    # row
    # SS[filename] = df.loc[:, 'SS'].astype(int).values.tolist()
    # TS[filename] = df.loc[:, 'TS1':'TS5'].mean(axis='columns').astype(int).values.tolist()
    # binary
    SS[filename] = (df.loc[:, 'SS'] > 4.0).astype(int).values.tolist()
    TS[filename] = (df.loc[:, 'TS1':'TS5'].mean(axis='columns') > 4.0).astype(int).values.tolist()
    # ternary
    # SS[filename] = df.loc[:, 'SS_ternary'].astype(int).values.tolist()
    # TS[filename] = df.loc[:, 'TS_ternary'].astype(int).values.tolist()

# SP_scaled = utils.dict_standardize(SP, vid)
# SP_binary = personality_binarization(SP_scaled, vid)
# SP_cluster = utils.clustering(SP_scaled, vid, n_clusters=4)

with open(feature_path + f'Hazumi{version}_features.pkl', mode='wb') as f:
    pickle.dump((SS, TS, SP, TP, text, audio, visual, vid), f)

  0%|          | 0/28 [00:00<?, ?it/s]

100%|██████████| 28/28 [00:35<00:00,  1.26s/it]


In [8]:
# 1911
version = "1911"
dumpfile_path = f'../data/Hazumi/Hazumi{version}/dumpfiles/*'
files = glob.glob(dumpfile_path)

text = {}
audio = {}
visual = {} 

SP = {}
TP = {}

SS = {}
TS = {}

vid = []

for file_path in tqdm(sorted(files)):
    filename = os.path.basename(file_path).split('.', 1)[0]
    df = pd.read_csv(file_path)
    start = df['start(exchange)[ms]'].values.tolist()

    vid.append(filename)
    text[filename] = extract_sentence(version, filename, start)
    audio[filename] = df.loc[:, 'pcm_RMSenergy_sma_max':'F0_sma_de_kurtosis'].values.tolist()
    visual[filename] = df.loc[:, '17_acceleration_max':'AU45_c_mean'].values.tolist()

    SP[filename] = get_personality(version, filename)
    TP[filename] = get_third_personality(version, filename)

    # row
    # SS[filename] = df.loc[:, 'SS'].astype(int).values.tolist()
    # TS[filename] = df.loc[:, 'TS1':'TS5'].values.tolist()
    # TS[filename] = df.loc[:, 'TS1':'TS5'].mean(axis='columns').values.tolist()
    # binary
    SS[filename] = (df.loc[:, 'SS'] > 4.0).astype(int).values.tolist()
    # TS[filename] = (df.loc[:, 'TS2'] > 4.0).astype(int).values.tolist()
    TS[filename] = (df.loc[:, 'TS1':'TS5'].mean(axis='columns') > 4.0).astype(int).values.tolist()
    # ternary
    # SS[filename] = df.loc[:, 'SS_ternary'].astype(int).values.tolist()
    # TS[filename] = df.loc[:, 'TS_ternary'].astype(int).values.tolist()

# SP_scaled = utils.dict_standardize(SP, vid)
# SP_binary = personality_binarization(SP_scaled, vid)
# SP_cluster = utils.clustering(SP_scaled, vid, n_clusters=4)

with open(feature_path + f'Hazumi{version}_features.pkl', mode='wb') as f:
    pickle.dump((SS, TS, SP, TP, text, audio, visual, vid), f)

  0%|          | 0/26 [00:00<?, ?it/s]

100%|██████████| 26/26 [00:38<00:00,  1.49s/it]


In [3]:
# 1911 small
version = "1911"
dumpfile_path = f'../data/Hazumi/Hazumi{version}/dumpfiles/*'
files = glob.glob(dumpfile_path)

text = {}
audio = {}
visual = {} 

SP = {}
TP = {}

SS = {}
TS = {}

vid = []

filenames = ["1911F2001", "1911F3001", "1911F4001", "1911F5001", "1911F6001", "1911F7002",
             "1911M2001", "1911M4001", "1911M5001", "1911M6001", "1911M7001"]

for file_path in tqdm(sorted(files)):
    filename = os.path.basename(file_path).split('.', 1)[0]
    if filename not in filenames:
        continue
    df = pd.read_csv(file_path)
    start = df['start(exchange)[ms]'].values.tolist()

    vid.append(filename)
    text[filename] = extract_sentence(version, filename, start)
    audio[filename] = df.loc[:, 'pcm_RMSenergy_sma_max':'F0_sma_de_kurtosis'].values.tolist()
    visual[filename] = df.loc[:, '17_acceleration_max':'AU45_c_mean'].values.tolist()

    SP[filename] = get_personality(version, filename)
    TP[filename] = get_third_personality(version, filename)

    # row
    # SS[filename] = df.loc[:, 'SS'].values.tolist()
    # TS[filename] = df.loc[:, 'TS1':'TS5'].mean(axis='columns').values.tolist()
    # binary
    SS[filename] = (df.loc[:, 'SS'] > 4.0).astype(int).values.tolist()
    TS[filename] = (df.loc[:, 'TS1':'TS5'].mean(axis='columns') > 4.0).astype(int).values.tolist()
    # ternary
    # SS[filename] = df.loc[:, 'SS_ternary'].astype(int).values.tolist()
    # TS[filename] = df.loc[:, 'TS_ternary'].astype(int).values.tolist()

# SP_scaled = utils.dict_standardize(SP, vid)
# SP_binary = personality_binarization(SP_scaled, vid)
# SP_cluster = utils.clustering(SP_scaled, vid, n_clusters=4)

with open(feature_path + f'Hazumi{version}_features_small.pkl', mode='wb') as f:
    pickle.dump((SS, TS, SP, TP, text, audio, visual, vid), f)

100%|██████████| 26/26 [00:22<00:00,  1.18it/s]


In [13]:
# 2010
version = "2010"
dumpfile_path = f'../data/Hazumi/Hazumi{version}/dumpfiles/*'
files = glob.glob(dumpfile_path)

text = {}
audio = {}
visual = {} 

SP = {}
TP = {}

SS = {}
TS = {}

vid = []

for file_path in tqdm(sorted(files)):
    filename = os.path.basename(file_path).split('.', 1)[0]
    df = pd.read_csv(file_path)
    start = df['start(exchange)'].values.tolist()

    vid.append(filename)
    text[filename] = extract_sentence(version, filename, start)
    audio[filename] = df.loc[:, 'pcm_RMSenergy_sma_max':'F0_sma_de_kurtosis\n'].values.tolist()
    visual[filename] = df.loc[:, '17_acceleration_max':'AU45_c_mean'].values.tolist()

    SP[filename] = get_personality(version, filename)
    TP[filename] = get_third_personality(version, filename)

    # row
    # TS[filename] = df.loc[:, 'TS1':'TS3'].mean(axis='columns').astype(int).values.tolist()
    # binary
    TS[filename] = (df.loc[:, 'TS1':'TS3'].mean(axis='columns') > 4.0).astype(int).values.tolist()
    # ternary
    # TS[filename] = df.loc[:, 'TS_ternary'].astype(int).values.tolist()

# SP_scaled = utils.dict_standardize(SP, vid)
# SP_binary = personality_binarization(SP_scaled, vid)

# SP_cluster = utils.clustering(SP_scaled, vid, n_clusters=4)

with open(feature_path + f'Hazumi{version}_features.pkl', mode='wb') as f:
    pickle.dump((SS, TS, SP, TP, text, audio, visual, vid), f)

100%|██████████| 33/33 [00:42<00:00,  1.29s/it]


In [5]:
version = "2012"
dumpfile_path = f'../data/Hazumi/Hazumi{version}/dumpfiles/*'
files = glob.glob(dumpfile_path)

text = {}
audio = {}
visual = {} 

SP = {}
TP = {}

SS = {}
TS = {}

vid = []

for file_path in tqdm(sorted(files)):
    filename = os.path.basename(file_path).split('.', 1)[0]
    df = pd.read_csv(file_path)
    start = df['start(exchange)'].values.tolist()

    vid.append(filename)
    text[filename] = extract_sentence(version, filename, start)
    audio[filename] = df.loc[:, 'pcm_RMSenergy_sma_max':'F0_sma_de_kurtosis\n'].values.tolist()
    visual[filename] = df.loc[:, '17_acceleration_max':'AU45_c_mean'].values.tolist()

    SP[filename] = get_personality(version, filename)
    TP[filename] = None

    # row
    # TS[filename] = df.loc[:, 'TS1':'TS3'].mean(axis='columns').astype(int).values.tolist()
    # binary
    TS[filename] = (df.loc[:, 'TS1':'TS3'].mean(axis="columns") > 4.0).astype(int).values.tolist()
    # ternary
    # TS[filename] = df.loc[:, 'TS_ternary'].astype(int).values.tolist()
    
# SP_scaled = utils.dict_standardize(SP, vid)
# SP_binary = personality_binarization(SP_scaled, vid)

# SP_cluster = utils.clustering(SP_scaled, vid, n_clusters=4)

with open(feature_path + f'Hazumi{version}_features.pkl', mode='wb') as f:
    pickle.dump((SS, TS, SP, TP, text, audio, visual, vid), f)

100%|██████████| 63/63 [01:23<00:00,  1.33s/it]


In [2]:
# All
with open('../data/Hazumi_features/Hazumi1712_features.pkl', mode='rb') as f:
    data1712 = pickle.load(f, encoding='utf-8')
    
with open('../data/Hazumi_features/Hazumi1902_features.pkl', mode='rb') as f:
    data1902 = pickle.load(f, encoding='utf-8')

with open('../data/Hazumi_features/Hazumi1911_features.pkl', mode='rb') as f:
    data1911 = pickle.load(f, encoding='utf-8')

with open('../data/Hazumi_features/Hazumi2010_features.pkl', mode='rb') as f:
    data2010 = pickle.load(f, encoding='utf-8')

with open('../data/Hazumi_features/Hazumi2012_features.pkl', mode='rb') as f:
    data2012 = pickle.load(f, encoding='utf-8')

data = {
    '1712': data1712,
    '1902': data1902,
    '1911': data1911,
    '2010': data2010, 
    '2012': data2012
}

def data_augmentation(versions):
    SS, TS, SP, TP = {}, {}, {}, {} 
    text, audio, visual, vid = {}, {}, {}, []
    for version in versions:
        SS.update(data[version][0])
        TS.update(data[version][1])
        SP.update(data[version][2])
        TP.update(data[version][3])
        text.update(data[version][4])
        audio.update(data[version][5])
        visual.update(data[version][6])
        vid.extend(data[version][7])
    return SS, TS, SP, TP, text, audio, visual, vid

SS, TS, SP, TP, text, audio, visual, vid = data_augmentation(['1712', '1902', '2010', '2012'])

with open(feature_path + 'Hazumiall_features.pkl', mode='wb') as f:
    pickle.dump((SS, TS, SP, TP, text, audio, visual, vid), f)

In [24]:
def clustering(id, mode):
    age = int(id[5])
    gender = id[4]

    if mode == 0:
        res = 0
    elif mode == 1:
        # 性別
        if gender == 'F':
            res = 0
        else:
            res = 1
    elif mode == 2:
        # 年齢 40 <=, 40 >
        if age <= 4:
            res = 0
        else:
            res = 1
    elif mode == 3:
        # 年齢 30 <=, 50 <=, 50 > 
        if age <= 3:
            res = 0 
        elif age <= 5:
            res = 1
        else:
            res = 2
    elif mode == 4: 
        # 年齢 20, 30, 40, 50, 60, 70
        res = age - 2
    return res

In [31]:
data = data2012
Age = [0, 0, 0, 0, 0, 0]
Gender = [0, 0]
for i, x in data[1].items():
    age = clustering(i, 4)
    gender = clustering(i, 1)
    Age[age] += 1
    Gender[gender] += 1 

print(Gender)
print(Age)
print(sum(Age))

[34, 29]
[15, 19, 20, 8, 1, 0]
63
